In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# Divide the annotation file into multiple .txt files for each image
# Define the paths
annotation_path = "./grocery_annotations/annotation.txt"
image_dir = "./Images_Grocery/ShelfImages/"

# Open the annotation file
with open(annotation_path, 'r') as f:
    lines = f.readlines()

# Process each line
for line in lines:
    parts = line.split()
    image_name = parts[0]
    num_objects = int(parts[1])
    objects = [parts[i:i+5] for i in range(2, len(parts), 5)]

    # Create a new annotation file for this image
    with open(os.path.join(image_dir, image_name + '.txt'), 'w') as f:
        for obj in objects:
            # Write the object annotation to the file
            f.write(' '.join(obj) + '\n')

In [ ]:
import shutil

# Specify the source directory and target directories
source_dir = image_dir
img_train_dir = './images/train'
label_train_dir = './labels/train'
img_val_dir = './images/val'
label_val_dir = './labels/val'

# Create the target directories if they don't exist
os.makedirs(img_train_dir, exist_ok=True)
os.makedirs(label_train_dir, exist_ok=True)
os.makedirs(img_val_dir, exist_ok=True)
os.makedirs(label_val_dir, exist_ok=True)

# Initialize counters for image and label files
img_counter = 0
label_counter = 0

# Iterate over all files in the source directory
for filename in os.listdir(source_dir):
    # If the file is a '.JPG' file
    if filename.endswith('.JPG'):
        # If we haven't moved 15 image files yet, move it to the image validation directory
        if img_counter < 15:
            shutil.move(os.path.join(source_dir, filename), os.path.join(img_val_dir, filename))
            img_counter += 1
        # Otherwise, move it to the image training directory
        else:
            shutil.move(os.path.join(source_dir, filename), os.path.join(img_train_dir, filename))
    # If the file is a '.txt' file
    elif filename.endswith('.txt'):
        # If we haven't moved 15 label files yet, move it to the label validation directory
        if label_counter < 15:
            shutil.move(os.path.join(source_dir, filename), os.path.join(label_val_dir, filename))
            label_counter += 1
        # Otherwise, move it to the label training directory
        else:
            shutil.move(os.path.join(source_dir, filename), os.path.join(label_train_dir, filename))

In [14]:
import cv2


# Specify the directory containing your text files
directory = './Images_Grocery/labels/train' #Change this to val or train

for filename in os.listdir(directory):
    if filename.endswith('.JPG.txt'):
        # Remove the '.JPG' from the filename
        new_filename = filename.replace('.JPG', '')
        # Rename the file
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
        
img_dir = './Images_Grocery/images/train/'

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Load the corresponding image to get its dimensions
        img_filename = os.path.splitext(filename)[0] + '.JPG' # replace with your image file extension
        img = cv2.imread(os.path.join(img_dir, img_filename))
        img_height, img_width = img.shape[:2]
        #print(img_height, img_width)

        # # Read the bounding box coordinates from the text file
        with open(os.path.join(directory, filename), 'r') as file:
            lines = file.readlines()

        # Convert the bounding box coordinates to the YOLO format and rewrite the text file
        with open(os.path.join(directory, filename), 'w') as file:
            for line in lines:
                parts = line.split()
                # Convert the top-left coordinates to center coordinates
                x_center = (float(parts[0]) + float(parts[2]) / 2) / img_width
                y_center = (float(parts[1]) + float(parts[3]) / 2) / img_height
                # Normalize the width and height
                width = float(parts[2]) / img_width
                height = float(parts[3]) / img_height
                # Write the converted coordinates back to the file
                file.write(f'{parts[4]} {x_center} {y_center} {width} {height}\n')

In [5]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [15]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

# Train the model using the 'coco128.yaml' dataset for 3 epochs
results = model.train(data='Images_Grocery/data_set.yaml', epochs=10, batch=8)

# Perform object detection on an image using the model
results = model('D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\Mask YOLO\Images_Grocery\ShelfImages\C1_P02_N3_S2_1.jpg')

Ultralytics YOLOv8.1.5 🚀 Python-3.11.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=Images_Grocery/data_set.yaml, epochs=10, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\GroceryDetection\Images_Grocery\labels\train... 338 images, 0 backgrounds, 1 corrupt: 100%|██████████| 338/338 [00:00<00:00, 411.83it/s]

train: WARNING ⚠️ D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\GroceryDetection\Images_Grocery\images\train\C3_P07_N1_S6_1.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0197      1.0164      1.0329      1.0312      1.0148      1.0197      1.0197      1.0148      1.0164      1.0115      1.0181      1.0263]


train: New cache created: D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\GroceryDetection\Images_Grocery\labels\train.cache


val: Scanning D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\GroceryDetection\Images_Grocery\labels\val... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:01<00:00, 11.13it/s]


...

      10/10      1.65G       1.02      1.122      1.033         45        640: 100%|██████████| 43/43 [00:24<00:00,  1.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]

                   all         15        284      0.964      0.333      0.359      0.288



10 epochs completed in 0.101 hours.
Optimizer stripped from runs\detect\train26\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train26\weights\best.pt, 6.3MB

Validating runs\detect\train26\weights\best.pt...
Ultralytics YOLOv8.1.5 🚀 Python-3.11.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3007793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


                   all         15        284      0.239          1      0.367      0.295
                 pack1         15        250      0.577          1      0.949      0.763
                 pack2         15         16     0.0792          1     0.0804     0.0645
                 pack5         15         18       0.06          1     0.0711     0.0559
Speed: 1.9ms preprocess, 27.4ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs\detect\train26

image 1/1 D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\Mask YOLO\Images_Grocery\ShelfImages\C1_P02_N3_S2_1.jpg: 480x640 27 pack1s, 65.0ms
Speed: 8.5ms preprocess, 65.0ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)


In [16]:
# Evaluate the model's performance on the validation set
results = model.val()

Ultralytics YOLOv8.1.5 🚀 Python-3.11.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)


val: Scanning D:\Cursuri\Master\Anul II\Semestrul I\Computer Vision\Grocery Detection\GroceryDetection\Images_Grocery\labels\val.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:26<00:00, 13.14s/it]


                   all         15        284      0.313      0.375      0.363      0.287
                 pack1         15        250      0.803          1      0.903      0.719
                 pack2         15         16      0.136      0.125      0.115      0.089
                 pack5         15         18          0          0       0.07     0.0532
Speed: 2.2ms preprocess, 84.3ms inference, 0.1ms loss, 44.8ms postprocess per image
Results saved to runs\detect\train262
